# **1. Perkenalan Dataset**


Tahap pertama, Anda harus mencari dan menggunakan dataset dengan ketentuan sebagai berikut:

1. **Sumber Dataset**:  
   #Dataset dapat diperoleh dari berbagai sumber, seperti public repositories (*Kaggle*, *UCI ML Repository*, *Open Data*) atau data primer yang Anda kumpulkan sendiri.
   Kaggle link: https://www.kaggle.com/datasets/redwankarimsony/heart-disease-data?resource=download


# **2. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning atau deep learning.

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer




ModuleNotFoundError: No module named 'pandas'

# **3. Memuat Dataset**

Pada tahap ini, Anda perlu memuat dataset ke dalam notebook. Jika dataset dalam format CSV, Anda bisa menggunakan pustaka pandas untuk membacanya. Pastikan untuk mengecek beberapa baris awal dataset untuk memahami strukturnya dan memastikan data telah dimuat dengan benar.

Jika dataset berada di Google Drive, pastikan Anda menghubungkan Google Drive ke Colab terlebih dahulu. Setelah dataset berhasil dimuat, langkah berikutnya adalah memeriksa kesesuaian data dan siap untuk dianalisis lebih lanjut.

Jika dataset berupa unstructured data, silakan sesuaikan dengan format seperti kelas Machine Learning Pengembangan atau Machine Learning Terapan

In [ ]:
dataset_path = '../heart.csv'

try:
    # Membaca dataset
    df = pd.read_csv(dataset_path)
    
    # Menampilkan 5 baris pertama untuk pengecekan
    print("Dataset berhasil dimuat!")
    print(f"Ukuran dataset: {df.shape[0]} baris, {df.shape[1]} kolom")
    display(df.head())

except FileNotFoundError:
    print(f"Error: File tidak ditemukan di {dataset_path}.")
    print("Pastikan path file sudah benar dan file sudah di-upload.")

# **4. Exploratory Data Analysis (EDA)**

Pada tahap ini, Anda akan melakukan **Exploratory Data Analysis (EDA)** untuk memahami karakteristik dataset.

Tujuan dari EDA adalah untuk memperoleh wawasan awal yang mendalam mengenai data dan menentukan langkah selanjutnya dalam analisis atau pemodelan.

In [ ]:
# 1. Cek Informasi Dasar (Tipe Data & Null)
print("=== Info Dataset ===")
df.info()

In [ ]:
# 2. Cek Statistik Deskriptif (Mean, Min, Max, dll)
print("\n=== Statistik Deskriptif ===")
display(df.describe())

In [ ]:
# 3. Cek Missing Values
print("\n=== Cek Missing Values ===")
print(df.isnull().sum())

In [ ]:
# 4. Visualisasi Target (Apakah seimbang atau imbalance?)
print("\n=== Nama Kolom ===")
print(df.columns)

target_col = 'num' # Target [0=no heart disease; 1,2,3,4 = stages of heart disease ]
#
if target_col in df.columns:
    plt.figure(figsize=(6, 4))
    sns.countplot(x=target_col, data=df, palette='viridis')
    plt.title('Distribusi Label Target (0=no heart disease; 1,2,3,4 = stages of heart disease)')
    plt.show()
else:
    print(f"Kolom '{target_col}' tidak ditemukan. Mohon cek nama kolom output.")

In [ ]:
# 5. Visualisasi Korelasi (Heatmap)
plt.figure(figsize=(12, 10))
correlation_matrix = df.corr(numeric_only=True)
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Heatmap Korelasi Antar Fitur')
plt.show()

# **5. Data Preprocessing**

In [ ]:
# A. CLEANING AWAL
# ----------------
df_clean = df.copy()

# 1. Ubah Target menjadi Binary (0 = Sehat, 1 = Sakit Jantung)
# Nilai 1, 2, 3, 4 dianggap '1' (Sakit)
df_clean['num'] = df_clean['num'].apply(lambda x: 1 if x > 0 else 0)

# 2. Buang kolom yang tidak perlu atau terlalu banyak missing values (>50%)
# 'id', 'dataset': Identitas (tidak guna untuk prediksi)
# 'ca', 'thal', 'slope': Missing value terlalu banyak (lihat hasil EDA)
cols_to_drop = ['id', 'dataset', 'ca', 'thal', 'slope']
df_clean = df_clean.drop(columns=cols_to_drop)

print("Ukuran dataset setelah cleaning awal:", df_clean.shape)

In [ ]:
# B. DATA SPLITTING
# -----------------
X = df_clean.drop(columns=['num'])
y = df_clean['num']

# Split 80% Train, 20% Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Jumlah Data Train: {X_train.shape[0]}")
print(f"Jumlah Data Test: {X_test.shape[0]}")

In [ ]:
# C. MEMBUAT PIPELINE PREPROCESSING
# ---------------------------------
numeric_features = ['age', 'trestbps', 'chol', 'thalch', 'oldpeak']
categorical_features = ['sex', 'cp', 'fbs', 'restecg', 'exang']

# Pipeline untuk Data Numerik:
# 1. Isi missing value dengan Median
# 2. Standarisasi skala (StandardScaler)
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Pipeline untuk Data Kategorikal:
# 1. Isi missing value dengan Modus (most_frequent)
# 2. Ubah teks jadi angka (OneHotEncoder)
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Gabungkan keduanya
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [ ]:
# D. TERAPKAN PIPELINE
# --------------------
# Fit hanya pada training data untuk mencegah data leakage
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# Ambil nama fitur baru setelah OneHotEncoding (untuk keperluan display saja)
# Ini agak tricky, tapi berguna untuk melihat hasil
ohe_feature_names = preprocessor.named_transformers_['cat']['encoder'].get_feature_names_out(categorical_features)
all_feature_names = numeric_features + list(ohe_feature_names)

# Konversi kembali ke DataFrame agar enak dilihat
X_train_df = pd.DataFrame(X_train_processed, columns=all_feature_names)
X_test_df = pd.DataFrame(X_test_processed, columns=all_feature_names)

print("\n=== Contoh Data Siap Latih (X_train) ===")
display(X_train_df.head())


In [ ]:
# E. SIMPAN DATASET HASIL PREPROCESSING
y_train_reset = y_train.reset_index(drop=True)
y_test_reset = y_test.reset_index(drop=True)

# Masukkan kembali target ke dataframe
X_train_df['num'] = y_train_reset
X_test_df['num'] = y_test_reset

# Simpan ke file CSV
X_train_df.to_csv('train_processed.csv', index=False)
X_test_df.to_csv('test_processed.csv', index=False)

print("\nDataset hasil preprocessing berhasil disimpan!")
print("File: 'train_processed.csv' dan 'test_processed.csv' telah dibuat.")